In [0]:
token = dbutils.widgets.get("Token_app")
print(token)

In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)
#token = dbutils.widgets.get("Token_app")
token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get(codigo_dataset, limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))

query="""
Select numero_del_contrato,numero_de_proceso
limit 30000000
"""


results = client.get(codigo_dataset, query=query)
 

In [0]:
results_df = spark.createDataFrame(results)

print(results_df.schema)
 

In [0]:
results_df

In [0]:
results_df.write.format("delta").mode("overwrite").save("/main.diplomado_datos.ids_contratos-_procesos")

In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = "codigo_dataset"
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))


In [0]:
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co", str(token), timeout=100)
 

In [0]:
import time
# 3. Parámetros de paginación
limit = 500000
offset = 0
write_mode = "overwrite"
reintentos=5

print(f"Iniciando carga por lotes para el dataset: {codigo_dataset}")

# 4. Bucle para obtener y cargar los datos por lotes
while True:
    intentos = 0

    while intentos < reintentos:
        try:
            print(f"Obteniendo y cargando lote {offset}...")
            # Construye y ejecuta la consulta para el lote actual
            query = f"SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor, estado_del_proceso LIMIT {limit} OFFSET {offset}"
            results = client.get(codigo_dataset, query=query) 

            # Si la API no devuelve más registros, se termina el bucle
            if not results:
                print("Carga de datos finalizada.")
                break

            # Convierte el lote a un DataFrame de Spark y lo escribe en la tabla Delta
            spark.createDataFrame(results).write \
                .format("delta") \
                .mode(write_mode) \
                .option("overwriteSchema", "true") \
                .saveAsTable("main.diplomado_datos.ids_contratos_procesos")

            print(f"Lote de {len(results)} registros desde offset {offset} cargado.")

            # Se cambia a modo 'append' para las siguientes iteraciones y se incrementa el offset
            write_mode = "append"
            offset += limit
            break

        except Exception as e:
            intentos +=1
            print("Error al obtener o cargar el lote:", e)
            print(f"Intento {intentos} de {reintentos}...")
            time.sleep(20)
    else:
        print("Se alcanzó el número máximo de intentos. Terminando la carga.")
        break
     

In [0]:
# Databricks notebook source
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata
 
#client = Socrata("www.datos.gov.co", None)
 
token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))
 
query="""
Select numero_del_contrato,numero_de_proceso, nit_de_la_entidad, documento_proveedor
limit 1000000
"""
 
 
results = client.get(codigo_dataset, query=query)
 
 
# COMMAND ----------
 
results_df = spark.createDataFrame(results)
 
print(results_df.schema)
 
# COMMAND ----------
 
results_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.diplomado_datos.ids_contratos_procesos")

In [0]:
limit = 10000
offset = 0
write_mode = "overwrite"

print(f'Iniciando la extraccion de datos del dataset {codigo_dataset}')

while True:
    print(f'Obteniendo registros desde el offset {offset}')
    query = f"""
    SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor
    LIMIT {limit}
    OFFSET {offset}
    """
    try:
        results = client.get(codigo_dataset, query=query)
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

    if not results:
        break

    spark.createDataFrame(results).write \
        .format("delta") \
        .mode(write_mode) \
        .saveAsTable("main.diplomado_datos.ids_contratos_procesos")
    write_mode = "append"
    offset += limit

# Databricks notebook source
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))

query="""
Select numero_del_contrato,numero_de_proceso, nit_de_la_entidad, documento_proveedor
limit 1000000
"""

try:
    results = client.get(codigo_dataset, query=query)
except Exception as e:
    print(f"Error fetching data: {e}")
    results = []

# COMMAND ----------

if results:
    results_df = spark.createDataFrame(results)
    print(results_df.schema)

    # COMMAND ----------

    results_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("main.diplomado_datos.ids_contratos_procesos")
else:
    print("No data fetched.")

In [0]:
codigo_dataset = "rpmr-utcd"  # o el ID real del dataset


In [0]:
limit = 10000
offset = 0
write_mode = "overwrite"

print(f'Iniciando la extraccion de datos del dataset {codigo_dataset}')

while True:
    print(f'Obteniendo registros desde el offset {offset}')
    query = f"""
    SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad, documento_proveedor
    LIMIT {limit}
    OFFSET {offset}
    """
    try:
        results = client.get(codigo_dataset, query=query)
    except Exception as e:
        print(f"Error fetching data: {e}")
        break

    if not results:
        break

    spark.createDataFrame(results).write \
        .format("delta") \
        .mode(write_mode) \
        .saveAsTable("main.diplomado_datos.ids_contratos_procesos")
    write_mode = "append"
    offset += limit

# Databricks notebook source
#!/usr/bin/env python
import pandas as pd
from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")
# Example authenticated client (needed for non-public datasets):
client = Socrata("www.datos.gov.co",
                  str(token))

query="""
Select numero_del_contrato,numero_de_proceso, nit_de_la_entidad, documento_proveedor
limit 1000000
"""

try:
    results = client.get(codigo_dataset, query=query)
except Exception as e:
    print(f"Error fetching data: {e}")
    results = []

# COMMAND ----------

if results:
    results_df = spark.createDataFrame(results)
    print(results_df.schema)

    # COMMAND ----------

    results_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("main.diplomado_datos.ids_contratos_procesos")
else:
    print("No data fetched.")

In [0]:
codigo_dataset = "rpmr-utcd"  # o el ID real del dataset
